In [131]:
import re
import requests
from bs4 import BeautifulSoup
import networkx as nx

G = nx.Graph()

In [142]:
page = requests.get("http://dtu-graph.lyxx.fr/data/class.html")
soup = BeautifulSoup(page.content, "html.parser")

for i in soup.findAll("tr"):
    #fetch departements
    if "colspan" in (str)(list(i.children)[1]):
        departement = list(i.children)[1].text.strip()
    #fetch class
    else:
        classnumber = i.text.strip().split('\n')[0][:5]
        classname = i.text.strip().split('\n')[0][8:]
        langage, ects = [s.strip() for s in i.text.strip().split('\n')[1].split('|')[0:2]]
        ects = (float)(ects.split(' ')[0])

        #get schedule. First dimension : different option, second dimension : all the class to follow for this option
        res = ""
        for j in list(i.children)[3].findAll("span")[1:]:
            if j.text.strip() != "":
                res += j.text.strip()
        res = res.split("or")
        for j in range(len(res)):
            res[j] = re.findall("(January|June|July|August|Spring|[EF][1-5][AB]?)", res[j])
        G.add_node(classnumber, name=classname, langage=langage, ects=ects, departement=departement, date=res)

In [152]:
page = requests.get("http://dtu-graph.lyxx.fr/data/ranking.html")
soup = BeautifulSoup(page.content, "html.parser")

#fetch class ratings
for i in soup.findChild("tbody").findAll("tr"):
    classnum = list(i.children)[0].text
    #some class might not exist anymore
    try:
        G.nodes[classnum]["avgGrade"] = list(i.children)[2].text
        G.nodes[classnum]["passed"] = list(i.children)[4].text
        G.nodes[classnum]["rating"] = list(i.children)[5].text
        G.nodes[classnum]["workload"] = list(i.children)[6].text
    except:pass
    